In [1]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras import optimizers
import numpy as np
from keras.layers.core import Lambda
from keras import backend as K
from keras.optimizers import SGD
from keras import regularizers

import tensorflow as tf
from keras import backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
K.set_session(sess)

from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten
from keras.layers import Conv2D,MaxPooling2D,ZeroPadding2D
from keras import models
from keras import layers
from keras.models import Model
import matplotlib.pyplot as plt
from keras.models import Sequential

from keras.callbacks import TensorBoard

C:\Users\dell\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#import data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)
 
weight_decay = 0.0005
nb_epoch=200
batch_size=128

In [3]:
from keras.models import load_model
model = load_model('0.8ent_b_cifar10vgg16_5_1.h5')
model.load_weights('0.8ent_b_cifar10vgg16_5_1_weights.h5')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1_input (InputLayer)  (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 64)        0         
__________

In [4]:
tensor =[]
for i in range(0,128):
   
    u=model.get_layer('conv2d_4').get_weights()[0][:,:,:,i].squeeze()
    v=np.mean(u, axis=1)
    #print(v)
    vT=v.T
    D=np.cov(vT)
    try:                 
        invD=np.linalg.inv(D)
        #t=np.mean(v,axis=1)
        a=[]

        for j in range(0,2): #表示filter的大小
            for k in range(j+1,3):
               #if (j!=k) and (j<k):
                tp=v[j]-v[k]
                d=np.sqrt(abs(np.dot(np.dot(tp,invD),tp.T)))
                a.append(d)
            
        print(i,a)
        tensor.append(a)
    except:
        print("不可逆")
print(tensor)

0 [11.317330198587818, 40.18912508750436, 35.133658431381]
1 [11.368109942889792, 15.98829962167586, 14.694114534828621]
2 [23.273253092782742, 17.510516583008037, 3.803675710884699]
3 [21.952739258767686, 33.63080155478253, 19.092064965748783]
4 [6.100081602316809, 16.160523725558853, 14.702702742218024]
5 [3.6487042413281507, 8.170473828367138, 11.896150746205537]
6 [15.452362165146438, 31.669089617808154, 19.548816233842953]
7 [3.3438065069539946, 4.972126922842586, 11.81860206063575]
8 [43.47361966135982, 26.618113545039105, 10.355292665185308]
9 [19.003342685791953, 15.276291867832363, 15.350305528116323]
10 [13.643831206153008, 12.989342952156996, 12.765431219378767]
11 [6.024644030743565, 15.029895499380736, 5.346327355328902]
12 [36.47754378151908, 21.186782104573012, 25.17772241511781]
13 [17.756974716999476, 23.84407443204569, 7.271462734783472]
14 [17.468563563353428, 14.155726341010451, 16.343239945864486]
15 [12.511409444853452, 8.756648834533696, 1.8983128196684138]
16 [2

In [5]:
entt=[]
for i in range (0,128):
    
    data=tensor[i]
    data0=np.array(data)
    print('四舍五入，精确到个位\n',np.round(data0,decimals=-2))
    data1=np.round(data0,decimals=-1)

    data1_value_list=set([data1[i] for i in range (data1.shape[0])])
    print(data1_value_list)
    ent=0.0
    for data1_value in data1_value_list:
        p=float(data1[data1==data1_value].shape[0])/data1.shape[0]
        print(p)
        logp=np.log2(p)
        ent-=p*logp
        print(ent)   
    entt.append(ent)
print(entt)



四舍五入，精确到个位
 [0. 0. 0.]
{40.0, 10.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{20.0, 30.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{20.0, 30.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{40.0, 10.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.

In [6]:
import heapq #获取list中最小的

f=int(len(entt)*0.7) #计算滤波器熵个数的80%
m=entt
max_number=heapq.nsmallest(f,m) #从m中找出最小的f个数，最大用nlargest
max_index=[]
for t in max_number:
    index=m.index(t)
    max_index.append(index)
    m[index]=float('-inf')
print(max_number)#输出最小的f个数
print(max_index)#输出对应索引

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896,

In [7]:
from kerassurgeon.operations import delete_layer,insert_layer,delete_channels
model_4 = delete_channels(model, model.layers[13],max_index)
model_4.summary()
model_4.save('8.7.6ent_b_cifar10vgg16_4.h5')
model_4.save_weights('8.7.6ent_b_cifar10vgg16__weights.h5',overwrite=True)

Deleting 89/128 channels from layer: conv2d_4
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1_input (InputLayer)  (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
activation_2 (Activation)    (

In [8]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model_4.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])
 
train_history=model_4.fit(x_train,y_train,epochs=200, batch_size=128,
             validation_split=0.1, verbose=1)

model_4.save('8.7.6ent_b_cifar10vgg16_4_1.h5')
model_4.save_weights('8.7.6ent_b_cifar10vgg16_4_1_weights.h5',overwrite=True)

Train on 45000 samples, validate on 5000 samples
Epoch 1/200
45000/45000 [==============================] - 28s 619us/step - loss: 0.8905 - acc: 0.7957 - val_loss: 0.7688 - val_acc: 0.8356
Epoch 2/200
45000/45000 [==============================] - 19s 424us/step - loss: 0.7561 - acc: 0.8354 - val_loss: 0.7582 - val_acc: 0.8336
Epoch 3/200
45000/45000 [==============================] - 19s 426us/step - loss: 0.7374 - acc: 0.8428 - val_loss: 0.7916 - val_acc: 0.8292
Epoch 4/200
45000/45000 [==============================] - 19s 428us/step - loss: 0.7286 - acc: 0.8486 - val_loss: 0.8383 - val_acc: 0.8130
Epoch 5/200
45000/45000 [==============================] - 19s 430us/step - loss: 0.6984 - acc: 0.8575 - val_loss: 0.8522 - val_acc: 0.8164
Epoch 6/200
45000/45000 [==============================] - 19s 427us/step - loss: 0.7054 - acc: 0.8566 - val_loss: 0.7795 - val_acc: 0.8348
Epoch 7/200
45000/45000 [==============================] - 19s 430us/step - loss: 0.6934 - acc: 0.8609 - val_lo

45000/45000 [==============================] - 21s 470us/step - loss: 0.6497 - acc: 0.8831 - val_loss: 0.7795 - val_acc: 0.8496
Epoch 59/200
45000/45000 [==============================] - 21s 471us/step - loss: 0.6516 - acc: 0.8816 - val_loss: 0.7451 - val_acc: 0.8534
Epoch 60/200
45000/45000 [==============================] - 21s 470us/step - loss: 0.6500 - acc: 0.8818 - val_loss: 0.7256 - val_acc: 0.8638
Epoch 61/200
45000/45000 [==============================] - 21s 475us/step - loss: 0.6527 - acc: 0.8809 - val_loss: 0.7777 - val_acc: 0.8528
Epoch 62/200
45000/45000 [==============================] - 21s 470us/step - loss: 0.6476 - acc: 0.8828 - val_loss: 0.7725 - val_acc: 0.8540
Epoch 63/200
45000/45000 [==============================] - 21s 472us/step - loss: 0.6462 - acc: 0.8855 - val_loss: 0.6971 - val_acc: 0.8708
Epoch 64/200
45000/45000 [==============================] - 21s 471us/step - loss: 0.6430 - acc: 0.8845 - val_loss: 0.7326 - val_acc: 0.8632
Epoch 65/200
45000/45000 [

45000/45000 [==============================] - 21s 473us/step - loss: 0.6252 - acc: 0.8913 - val_loss: 0.7886 - val_acc: 0.8448
Epoch 117/200
45000/45000 [==============================] - 21s 474us/step - loss: 0.6312 - acc: 0.8895 - val_loss: 0.8136 - val_acc: 0.8452
Epoch 118/200
45000/45000 [==============================] - 21s 469us/step - loss: 0.6356 - acc: 0.8870 - val_loss: 0.7642 - val_acc: 0.8508
Epoch 119/200
45000/45000 [==============================] - 21s 470us/step - loss: 0.6287 - acc: 0.8895 - val_loss: 0.7409 - val_acc: 0.8626
Epoch 120/200
45000/45000 [==============================] - 21s 473us/step - loss: 0.6338 - acc: 0.8882 - val_loss: 0.7052 - val_acc: 0.8700
Epoch 121/200
45000/45000 [==============================] - 21s 472us/step - loss: 0.6314 - acc: 0.8878 - val_loss: 0.7643 - val_acc: 0.8564
Epoch 122/200
45000/45000 [==============================] - 21s 472us/step - loss: 0.6327 - acc: 0.8888 - val_loss: 0.7690 - val_acc: 0.8466
Epoch 123/200
45000/

45000/45000 [==============================] - 21s 475us/step - loss: 0.6201 - acc: 0.8919 - val_loss: 0.8304 - val_acc: 0.8422
Epoch 174/200
45000/45000 [==============================] - 21s 472us/step - loss: 0.6140 - acc: 0.8932 - val_loss: 0.8494 - val_acc: 0.8342
Epoch 175/200
45000/45000 [==============================] - 21s 472us/step - loss: 0.6133 - acc: 0.8922 - val_loss: 0.8052 - val_acc: 0.8408
Epoch 176/200
45000/45000 [==============================] - 21s 473us/step - loss: 0.6224 - acc: 0.8910 - val_loss: 0.7503 - val_acc: 0.8550
Epoch 177/200
45000/45000 [==============================] - 21s 471us/step - loss: 0.6321 - acc: 0.8905 - val_loss: 0.7056 - val_acc: 0.8726
Epoch 178/200
45000/45000 [==============================] - 21s 472us/step - loss: 0.6199 - acc: 0.8924 - val_loss: 0.7678 - val_acc: 0.8512
Epoch 179/200
45000/45000 [==============================] - 21s 471us/step - loss: 0.6219 - acc: 0.8913 - val_loss: 0.7302 - val_acc: 0.8576
Epoch 180/200
45000/

In [9]:
import functools
import keras
top1_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=1) #top-1精度

top1_acc.__name__ = 'top1_acc'

model_4.compile(loss='categorical_crossentropy',
             optimizer='adam',metrics=['accuracy','top_k_categorical_accuracy',top1_acc])

model_4.evaluate(x_test,y_test,verbose=0)

[0.7724098143577576, 0.8453, 0.9897, 0.8453]

In [10]:
score=model_4.evaluate(x_test,y_test,verbose=0)
print("测试损失为：{:.4f}".format(score[0]))
print("测试准确率为：{:.4f}".format(score[1]))

测试损失为：0.7724
测试准确率为：0.8453


In [11]:
tensor =[]
for i in range(0,128):
   
    u=model_4.get_layer('conv2d_3').get_weights()[0][:,:,:,i].squeeze()
    v=np.mean(u, axis=1)
    #print(v)
    vT=v.T
    D=np.cov(vT)
    try:                 
        invD=np.linalg.inv(D)
        #t=np.mean(v,axis=1)
        a=[]

        for j in range(0,2): #表示filter的大小
            for k in range(j+1,3):
               #if (j!=k) and (j<k):
                tp=v[j]-v[k]
                d=np.sqrt(abs(np.dot(np.dot(tp,invD),tp.T)))
                a.append(d)
            
        print(i,a)
        tensor.append(a)
    except:
        print("不可逆")
print(tensor)

0 [11.308420910609762, 26.478920307909224, 26.34537458557846]
1 [20.476583737708367, 1.2002084233163064, 17.873948853270484]
2 [10.90440759699966, 46.57610614832921, 28.41881770907596]
3 [5.96243087440583, 7.311813257036446, 16.195286472445016]
4 [14.946310046934284, 11.08312555825844, 9.57033178210749]
5 [28.55270010611272, 11.739058330793592, 23.135302990927965]
6 [14.222955144786663, 35.72191217397358, 34.1287438431454]
7 [8.988552548634049, 27.313881260842493, 8.449666230533898]
8 [7.611988519607803, 11.359744835963072, 12.893112737871805]
9 [13.70709408527973, 7.42596945807569, 11.430962950959383]
10 [11.61127679170213, 8.71732999661579, 10.022331075636384]
11 [11.564025407125245, 3.211935496179007, 8.494344989969754]
12 [15.58527238616973, 34.915917952225364, 34.59493806999123]
13 [74.80777215001027, 27.853226681248383, 15.445392274626432]
14 [11.697110411843044, 19.650158737356815, 5.590659625147151]
15 [19.708627971465674, 8.749008967271282, 8.15106594744266]
16 [20.25527438009

In [12]:
entt=[]
for i in range (0,128):
    
    data=tensor[i]
    data0=np.array(data)
    print('四舍五入，精确到个位\n',np.round(data0,decimals=-2))
    data1=np.round(data0,decimals=-1)

    data1_value_list=set([data1[i] for i in range (data1.shape[0])])
    print(data1_value_list)
    ent=0.0
    for data1_value in data1_value_list:
        p=float(data1[data1==data1_value].shape[0])/data1.shape[0]
        print(p)
        logp=np.log2(p)
        ent-=p*logp
        print(ent)   
    entt.append(ent)
print(entt)

四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 30.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 50.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{40.0, 10.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 30.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0}
1.0
0.0
四舍五入，精确到个位

{20.0, 30.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{50.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [200. 300. 100.]
{140.0, 210.0, 300.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0}
1.0

In [13]:
import heapq #获取list中最小的

f=int(len(entt)*0.7) #计算滤波器熵个数的80%
m=entt
max_number=heapq.nsmallest(f,m) #从m中找出最小的f个数，最大用nlargest
max_index=[]
for t in max_number:
    index=m.index(t)
    max_index.append(index)
    m[index]=float('-inf')
print(max_number)#输出最小的f个数
print(max_index)#输出对应索引

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.91

In [14]:
from kerassurgeon.operations import delete_layer,insert_layer,delete_channels
model_3 = delete_channels(model_4, model_4.layers[9],max_index)
model_3.summary()
model_3.save('8.7.6ent_b_cifar10vgg16_3.h5')
model_3.save_weights('8.7.6ent_b_cifar10vgg16_3_weights.h5',overwrite=True)

Deleting 89/128 channels from layer: conv2d_3
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1_input (InputLayer)  (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
activation_2 (Activation)    (

In [15]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model_3.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])
 
train_history=model_3.fit(x_train,y_train,epochs=200, batch_size=128,
             validation_split=0.1, verbose=1)

model_3.save('8.7.6ent_b_cifar10vgg16_3_1.h5')
model_3.save_weights('8.7.6ent_b_cifar10vgg16_3_1_weights.h5',overwrite=True)

Train on 45000 samples, validate on 5000 samples
Epoch 1/200
45000/45000 [==============================] - 26s 584us/step - loss: 0.8678 - acc: 0.7959 - val_loss: 0.7408 - val_acc: 0.8378
Epoch 2/200
45000/45000 [==============================] - 20s 448us/step - loss: 0.7629 - acc: 0.8304 - val_loss: 1.0166 - val_acc: 0.7654
Epoch 3/200
45000/45000 [==============================] - 20s 446us/step - loss: 0.7622 - acc: 0.8331 - val_loss: 0.8050 - val_acc: 0.8206
Epoch 4/200
45000/45000 [==============================] - 20s 449us/step - loss: 0.7438 - acc: 0.8400 - val_loss: 0.7657 - val_acc: 0.8338
Epoch 5/200
45000/45000 [==============================] - 20s 449us/step - loss: 0.7314 - acc: 0.8434 - val_loss: 0.7381 - val_acc: 0.8406
Epoch 6/200
45000/45000 [==============================] - 20s 449us/step - loss: 0.7227 - acc: 0.8460 - val_loss: 0.7309 - val_acc: 0.8436
Epoch 7/200
45000/45000 [==============================] - 20s 454us/step - loss: 0.7183 - acc: 0.8472 - val_lo

Epoch 59/200
45000/45000 [==============================] - 20s 451us/step - loss: 0.6852 - acc: 0.8615 - val_loss: 0.7697 - val_acc: 0.8366
Epoch 60/200
45000/45000 [==============================] - 20s 453us/step - loss: 0.6840 - acc: 0.8611 - val_loss: 0.7611 - val_acc: 0.8350
Epoch 61/200
45000/45000 [==============================] - 20s 452us/step - loss: 0.6898 - acc: 0.8605 - val_loss: 0.7550 - val_acc: 0.8476
Epoch 62/200
45000/45000 [==============================] - 20s 447us/step - loss: 0.6913 - acc: 0.8612 - val_loss: 0.7397 - val_acc: 0.8500
Epoch 63/200
45000/45000 [==============================] - 20s 453us/step - loss: 0.6939 - acc: 0.8602 - val_loss: 0.7511 - val_acc: 0.8378
Epoch 64/200
45000/45000 [==============================] - 20s 453us/step - loss: 0.6958 - acc: 0.8589 - val_loss: 0.7139 - val_acc: 0.8544
Epoch 65/200
45000/45000 [==============================] - 20s 452us/step - loss: 0.6925 - acc: 0.8621 - val_loss: 0.8317 - val_acc: 0.8246
Epoch 66/200


Epoch 117/200
45000/45000 [==============================] - 20s 450us/step - loss: 0.6771 - acc: 0.8629 - val_loss: 1.0303 - val_acc: 0.7678
Epoch 118/200
45000/45000 [==============================] - 20s 453us/step - loss: 0.6780 - acc: 0.8630 - val_loss: 0.7077 - val_acc: 0.8574
Epoch 119/200
45000/45000 [==============================] - 20s 451us/step - loss: 0.6789 - acc: 0.8645 - val_loss: 0.6949 - val_acc: 0.8610
Epoch 120/200
45000/45000 [==============================] - 20s 453us/step - loss: 0.6808 - acc: 0.8660 - val_loss: 0.7202 - val_acc: 0.8544
Epoch 121/200
45000/45000 [==============================] - 20s 442us/step - loss: 0.6714 - acc: 0.8660 - val_loss: 0.7025 - val_acc: 0.8562
Epoch 122/200
45000/45000 [==============================] - 20s 434us/step - loss: 0.6767 - acc: 0.8653 - val_loss: 0.7418 - val_acc: 0.8488
Epoch 123/200
45000/45000 [==============================] - 20s 452us/step - loss: 0.6743 - acc: 0.8669 - val_loss: 0.7762 - val_acc: 0.8352
Epoch 

45000/45000 [==============================] - 20s 450us/step - loss: 0.6608 - acc: 0.8695 - val_loss: 0.7629 - val_acc: 0.8422
Epoch 175/200
45000/45000 [==============================] - 20s 451us/step - loss: 0.6780 - acc: 0.8670 - val_loss: 0.7074 - val_acc: 0.8576
Epoch 176/200
45000/45000 [==============================] - 20s 452us/step - loss: 0.6690 - acc: 0.8670 - val_loss: 0.6947 - val_acc: 0.8596
Epoch 177/200
45000/45000 [==============================] - 20s 453us/step - loss: 0.6770 - acc: 0.8639 - val_loss: 0.8638 - val_acc: 0.8066
Epoch 178/200
45000/45000 [==============================] - 20s 453us/step - loss: 0.6709 - acc: 0.8665 - val_loss: 0.7349 - val_acc: 0.8450
Epoch 179/200
45000/45000 [==============================] - 20s 451us/step - loss: 0.6647 - acc: 0.8691 - val_loss: 0.7733 - val_acc: 0.8412
Epoch 180/200
45000/45000 [==============================] - 20s 452us/step - loss: 0.6700 - acc: 0.8684 - val_loss: 0.7795 - val_acc: 0.8326
Epoch 181/200
45000/

In [16]:
import functools
import keras
top1_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=1) #top-1精度

top1_acc.__name__ = 'top1_acc'

model_3.compile(loss='categorical_crossentropy',
             optimizer='adam',metrics=['accuracy','top_k_categorical_accuracy',top1_acc])

model_3.evaluate(x_test,y_test,verbose=0)

[0.6963983918190002, 0.8629, 0.9935, 0.8629]

In [17]:
score=model_3.evaluate(x_test,y_test,verbose=0)
print("测试损失为：{:.4f}".format(score[0]))
print("测试准确率为：{:.4f}".format(score[1]))

测试损失为：0.6964
测试准确率为：0.8629


In [18]:
tensor =[]
for i in range(0,64):
   
    u=model_3.get_layer('conv2d_2').get_weights()[0][:,:,:,i].squeeze()
    v=np.mean(u, axis=1)
    #print(v)
    vT=v.T
    D=np.cov(vT)
    try:                 
        invD=np.linalg.inv(D)
        #t=np.mean(v,axis=1)
        a=[]

        for j in range(0,2): #表示filter的大小
            for k in range(j+1,3):
               #if (j!=k) and (j<k):
                tp=v[j]-v[k]
                d=np.sqrt(abs(np.dot(np.dot(tp,invD),tp.T)))
                a.append(d)
            
        print(i,a)
        tensor.append(a)
    except:
        print("不可逆")
print(tensor)

0 [9.573772026232913, 10.96541299623161, 12.776673446608358]
1 [22.405420774813305, 5.969915545839044, 19.414438802038752]
2 [8.708007018502192, 26.388167217715356, 1.8701771061713022]
3 [11.543836470002812, 17.628113005035058, 12.161449448162267]
4 [45.49184170617252, 7.205237287840316, 33.73041056333646]
5 [6.949131433840055, 4.202001400200648, 11.651645824926833]
6 [60.01505268740723, 55.95647428278658, 20.23519638927672]
7 [23.658302167724347, 8.344651077134447, 7.403995460487223]
8 [22.25741629283032, 33.18383290348912, 21.0102904402182]
9 [17.399949006849962, 5.7699290805405195, 6.941863242987976]
10 [8.543137983434653, 10.481809708738403, 14.811451452300018]
11 [21.97196228287795, 16.487734079036912, 32.3550582209408]
12 [4.047300780296161, 10.892429470493191, 5.736247866429198]
13 [20.825539052793975, 43.333540915334346, 30.481851869280142]
14 [16.10346476845839, 6.390828164375826, 16.063155580196675]
15 [9.812689627497713, 4.642228745284021, 8.591012706035409]
16 [8.8651589379

In [19]:
entt=[]
for i in range (0,64):
    
    data=tensor[i]
    data0=np.array(data)
    print('四舍五入，精确到个位\n',np.round(data0,decimals=-2))
    data1=np.round(data0,decimals=-1)

    data1_value_list=set([data1[i] for i in range (data1.shape[0])])
    print(data1_value_list)
    ent=0.0
    for data1_value in data1_value_list:
        p=float(data1[data1==data1_value].shape[0])/data1.shape[0]
        print(p)
        logp=np.log2(p)
        ent-=p*logp
        print(ent)   
    entt.append(ent)
print(entt)

四舍五入，精确到个位
 [0. 0. 0.]
{10.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{50.0, 10.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.0566416671474375
0.3333333333333333
1.584962500721156
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [100. 100.   0.]
{60.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{20.0, 30.0}
0.6666666666666666
0.38997500048077083
0.33333333

In [20]:
import heapq #获取list中最小的

f=int(len(entt)*0.6) #计算滤波器熵个数的80%
m=entt
max_number=heapq.nsmallest(f,m) #从m中找出最小的f个数，最大用nlargest
max_index=[]
for t in max_number:
    index=m.index(t)
    max_index.append(index)
    m[index]=float('-inf')
print(max_number)#输出最小的f个数
print(max_index)#输出对应索引

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896]
[0, 10, 22, 25, 37, 38, 50, 53, 56, 58, 1, 3, 5, 6, 7, 8, 9, 11, 12, 14, 15, 16, 17, 20, 21, 23, 24, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36]


In [21]:
from kerassurgeon.operations import delete_layer,insert_layer,delete_channels
model_2 = delete_channels(model_3, model_3.layers[5],max_index)
model_2.summary()
model_2.save('8.7.6ent_b_cifar10vgg16_2.h5')
model_2.save_weights('8.7.6ent_b_cifar10vgg16_2_weights.h5',overwrite=True)

Deleting 38/64 channels from layer: conv2d_2
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1_input (InputLayer)  (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 26)        15002     
_________________________________________________________________
activation_2 (Activation)    mu

In [22]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model_2.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])
 
train_history=model_2.fit(x_train,y_train,epochs=200, batch_size=128,
             validation_split=0.1, verbose=1)

model_2.save('8.7.6ent_b_cifar10vgg16_2_1.h5')
model_2.save_weights('8.7.6ent_b_cifar10vgg16_2_1_weights.h5',overwrite=True)

Train on 45000 samples, validate on 5000 samples
Epoch 1/200
45000/45000 [==============================] - 26s 577us/step - loss: 0.8387 - acc: 0.8063 - val_loss: 0.7749 - val_acc: 0.8252
Epoch 2/200
45000/45000 [==============================] - 19s 428us/step - loss: 0.7605 - acc: 0.8300 - val_loss: 0.7798 - val_acc: 0.8290
Epoch 3/200
45000/45000 [==============================] - 19s 428us/step - loss: 0.7487 - acc: 0.8359 - val_loss: 0.7232 - val_acc: 0.8416
Epoch 4/200
45000/45000 [==============================] - 19s 427us/step - loss: 0.7544 - acc: 0.8338 - val_loss: 0.8030 - val_acc: 0.8200
Epoch 5/200
45000/45000 [==============================] - 19s 426us/step - loss: 0.7462 - acc: 0.8375 - val_loss: 0.7388 - val_acc: 0.8358
Epoch 6/200
45000/45000 [==============================] - 19s 425us/step - loss: 0.7435 - acc: 0.8385 - val_loss: 0.8500 - val_acc: 0.8040
Epoch 7/200
45000/45000 [==============================] - 19s 423us/step - loss: 0.7274 - acc: 0.8454 - val_lo

Epoch 59/200
45000/45000 [==============================] - 19s 424us/step - loss: 0.7089 - acc: 0.8524 - val_loss: 0.7249 - val_acc: 0.8450
Epoch 60/200
45000/45000 [==============================] - 19s 428us/step - loss: 0.7105 - acc: 0.8516 - val_loss: 0.8077 - val_acc: 0.8204
Epoch 61/200
45000/45000 [==============================] - 19s 427us/step - loss: 0.7039 - acc: 0.8542 - val_loss: 0.8019 - val_acc: 0.8238
Epoch 62/200
45000/45000 [==============================] - 19s 427us/step - loss: 0.7046 - acc: 0.8525 - val_loss: 0.7181 - val_acc: 0.8472
Epoch 63/200
45000/45000 [==============================] - 19s 426us/step - loss: 0.7127 - acc: 0.8521 - val_loss: 0.7480 - val_acc: 0.8402
Epoch 64/200
45000/45000 [==============================] - 19s 424us/step - loss: 0.7081 - acc: 0.8525 - val_loss: 0.7196 - val_acc: 0.8494
Epoch 65/200
45000/45000 [==============================] - 19s 425us/step - loss: 0.7109 - acc: 0.8530 - val_loss: 0.6939 - val_acc: 0.8558
Epoch 66/200


Epoch 117/200
45000/45000 [==============================] - 19s 428us/step - loss: 0.7049 - acc: 0.8548 - val_loss: 0.8436 - val_acc: 0.8216
Epoch 118/200
45000/45000 [==============================] - 19s 426us/step - loss: 0.6889 - acc: 0.8582 - val_loss: 0.7211 - val_acc: 0.8452
Epoch 119/200
45000/45000 [==============================] - 19s 426us/step - loss: 0.6968 - acc: 0.8574 - val_loss: 0.7663 - val_acc: 0.8434
Epoch 120/200
45000/45000 [==============================] - 19s 425us/step - loss: 0.6990 - acc: 0.8571 - val_loss: 0.7379 - val_acc: 0.8402
Epoch 121/200
45000/45000 [==============================] - 19s 423us/step - loss: 0.6915 - acc: 0.8582 - val_loss: 0.7116 - val_acc: 0.8502
Epoch 122/200
45000/45000 [==============================] - 19s 423us/step - loss: 0.6968 - acc: 0.8566 - val_loss: 0.7059 - val_acc: 0.8552
Epoch 123/200
45000/45000 [==============================] - 19s 428us/step - loss: 0.6885 - acc: 0.8597 - val_loss: 0.7093 - val_acc: 0.8574
Epoch 

Epoch 175/200
45000/45000 [==============================] - 19s 425us/step - loss: 0.6844 - acc: 0.8603 - val_loss: 0.7582 - val_acc: 0.8364
Epoch 176/200
45000/45000 [==============================] - 19s 428us/step - loss: 0.6863 - acc: 0.8597 - val_loss: 0.7466 - val_acc: 0.8408
Epoch 177/200
45000/45000 [==============================] - 19s 425us/step - loss: 0.6905 - acc: 0.8591 - val_loss: 0.7437 - val_acc: 0.8458
Epoch 178/200
45000/45000 [==============================] - 19s 427us/step - loss: 0.6925 - acc: 0.8586 - val_loss: 0.8099 - val_acc: 0.8210
Epoch 179/200
45000/45000 [==============================] - 19s 428us/step - loss: 0.6889 - acc: 0.8599 - val_loss: 0.7603 - val_acc: 0.8386
Epoch 180/200
45000/45000 [==============================] - 19s 427us/step - loss: 0.6947 - acc: 0.8573 - val_loss: 0.8104 - val_acc: 0.8236
Epoch 181/200
45000/45000 [==============================] - 19s 427us/step - loss: 0.6871 - acc: 0.8594 - val_loss: 0.8230 - val_acc: 0.8216
Epoch 

In [23]:
import functools
import keras
top1_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=1) #top-1精度

top1_acc.__name__ = 'top1_acc'

model_2.compile(loss='categorical_crossentropy',
             optimizer='adam',metrics=['accuracy','top_k_categorical_accuracy',top1_acc])

model_2.evaluate(x_test,y_test,verbose=0)

[0.8574253187179566, 0.8093, 0.9881, 0.8093]

In [24]:
score=model_2.evaluate(x_test,y_test,verbose=0)
print("测试损失为：{:.4f}".format(score[0]))
print("测试准确率为：{:.4f}".format(score[1]))

测试损失为：0.8574
测试准确率为：0.8093


In [25]:
tensor =[]
for i in range(0,64):
   
    u=model_2.get_layer('conv2d_1').get_weights()[0][:,:,:,i].squeeze()
    v=np.mean(u, axis=1)
    #print(v)
    vT=v.T
    D=np.cov(vT)
    try:                 
        invD=np.linalg.inv(D)
        #t=np.mean(v,axis=1)
        a=[]

        for j in range(0,2): #表示filter的大小
            for k in range(j+1,3):
               #if (j!=k) and (j<k):
                tp=v[j]-v[k]
                d=np.sqrt(abs(np.dot(np.dot(tp,invD),tp.T)))
                a.append(d)
            
        print(i,a)
        tensor.append(a)
    except:
        print("不可逆")
print(tensor)

0 [16.644323118959143, 1.5526349161416408, 1.9765302934397464]
1 [2.9154759474226504, 1.3228756555322954, 0.5590169943749475]
2 [3.146635656905295, 2.837096019037473, 4.692058983146991]
3 [2.5369909678440226, 10.207552639958413, 8.831261420326921]
4 [20.421903976501596, 17.228361406923703, 1.6817761124195998]
5 [2.327026933899001, 1.4675199349396213, 1.0847759483980834]
6 [1.9657425815319673, 2.8740712303467855, 2.042306123862451]
7 [2.227754615077702, 2.4980461114639176, 1.653594569415369]
8 [10.110114210598656, 10.218423295704296, 2.001953065453505]
9 [12.611061947657188, 2.0000011813576455, 1.2348743471297698]
10 [2.099106952968333, 1.9921721813136535, 2.692582403567252]
11 [5.586622317557722, 2.5101206283957294, 3.007197738343821]
12 [30.66874045788391, 21.8770303056471, 9.106411905099579]
13 [13.856441699170102, 0.0, 11.215700207848984]
14 [30.454622986357094, 8.326921014553404, 50.83351847297014]
15 [1.557491974297139, 1.25, 2.0223362109204297]
16 [1.8192224472284855, 1.507792260

In [26]:
entt=[]
for i in range (0,64):
    
    data=tensor[i]
    data0=np.array(data)
    print('四舍五入，精确到个位\n',np.round(data0,decimals=-2))
    data1=np.round(data0,decimals=-1)

    data1_value_list=set([data1[i] for i in range (data1.shape[0])])
    print(data1_value_list)
    ent=0.0
    for data1_value in data1_value_list:
        p=float(data1[data1==data1_value].shape[0])/data1.shape[0]
        print(p)
        logp=np.log2(p)
        ent-=p*logp
        print(ent)   
    entt.append(ent)
print(entt)

四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 20.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{0.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 20.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{0.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{0.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.3333333333333333
0.5283208335737187
0.6666666666666666
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{0.0}
1.0
0.0
四舍五入，精确到个位
 [0. 0. 0.]
{0.0, 10.0}
0.6666666666666666
0.38997500048077083
0.3333333333333333
0.9182958340544896
四舍五入，精确到个位
 [0. 0. 0.]
{10.0, 20.0, 30.0}
0.3333333333333333
0.5283208335737187
0.3333333333333333
1.05664166

In [27]:
import heapq #获取list中最小的

f=int(len(entt)*0.6) #计算滤波器熵个数的80%
m=entt
max_number=heapq.nsmallest(f,m) #从m中找出最小的f个数，最大用nlargest
max_index=[]
for t in max_number:
    index=m.index(t)
    max_index.append(index)
    m[index]=float('-inf')
print(max_number)#输出最小的f个数
print(max_index)#输出对应索引

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896, 0.9182958340544896]
[1, 2, 5, 6, 7, 10, 15, 16, 17, 18, 19, 20, 23, 25, 30, 32, 34, 35, 38, 40, 42, 46, 47, 48, 52, 54, 56, 57, 58, 63, 0, 3, 4, 8, 9, 11, 13, 21]


In [28]:
from kerassurgeon.operations import delete_layer,insert_layer,delete_channels
model_1 = delete_channels(model_2, model_2.layers[1],max_index)
model_1.summary()
model_1.save('8.7.6ent_b_cifar10vgg16_1.h5')
model_1.save_weights('8.7.6ent_b_cifar10vgg16_1_weights.h5',overwrite=True)

Deleting 38/64 channels from layer: conv2d_1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1_input (InputLayer)  (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 26)        728       
_________________________________________________________________
activation_1 (Activation)    multiple                  0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 26)        104       
_________________________________________________________________
dropout_1 (Dropout)          multiple                  0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 26)        6110      
_________________________________________________________________
activation_2 (Activation)    mu

In [29]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model_1.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])
 
train_history=model_1.fit(x_train,y_train,epochs=200, batch_size=128,
             validation_split=0.1, verbose=1)

model_1.save('8.7.6ent_b_cifar10vgg16_1_1.h5')
model_1.save_weights('8.7.6ent_b_cifar10vgg16_1_1_weights.h5',overwrite=True)

Train on 45000 samples, validate on 5000 samples
Epoch 1/200
45000/45000 [==============================] - 25s 558us/step - loss: 0.7411 - acc: 0.8365 - val_loss: 0.7183 - val_acc: 0.8462
Epoch 2/200
45000/45000 [==============================] - 18s 398us/step - loss: 0.7376 - acc: 0.8389 - val_loss: 0.9433 - val_acc: 0.7782
Epoch 3/200
45000/45000 [==============================] - 18s 399us/step - loss: 0.7291 - acc: 0.8435 - val_loss: 0.7213 - val_acc: 0.8400
Epoch 4/200
45000/45000 [==============================] - 18s 399us/step - loss: 0.7270 - acc: 0.8457 - val_loss: 1.2385 - val_acc: 0.6936
Epoch 5/200
45000/45000 [==============================] - 18s 401us/step - loss: 0.7296 - acc: 0.8423 - val_loss: 0.8813 - val_acc: 0.8004
Epoch 6/200
45000/45000 [==============================] - 18s 400us/step - loss: 0.7192 - acc: 0.8485 - val_loss: 0.7198 - val_acc: 0.8508
Epoch 7/200
45000/45000 [==============================] - 18s 399us/step - loss: 0.7171 - acc: 0.8492 - val_lo

Epoch 59/200
45000/45000 [==============================] - 18s 395us/step - loss: 0.7124 - acc: 0.8522 - val_loss: 0.7259 - val_acc: 0.8464
Epoch 60/200
45000/45000 [==============================] - 18s 397us/step - loss: 0.7062 - acc: 0.8549 - val_loss: 0.7888 - val_acc: 0.8302
Epoch 61/200
45000/45000 [==============================] - 18s 395us/step - loss: 0.7102 - acc: 0.8530 - val_loss: 0.8206 - val_acc: 0.8216
Epoch 62/200
45000/45000 [==============================] - 18s 395us/step - loss: 0.7035 - acc: 0.8548 - val_loss: 0.7515 - val_acc: 0.8362
Epoch 63/200
45000/45000 [==============================] - 18s 398us/step - loss: 0.7031 - acc: 0.8533 - val_loss: 0.7654 - val_acc: 0.8386
Epoch 64/200
45000/45000 [==============================] - 18s 399us/step - loss: 0.7114 - acc: 0.8514 - val_loss: 0.8122 - val_acc: 0.8238
Epoch 65/200
45000/45000 [==============================] - 18s 396us/step - loss: 0.7052 - acc: 0.8509 - val_loss: 0.7368 - val_acc: 0.8426
Epoch 66/200


45000/45000 [==============================] - 18s 396us/step - loss: 0.7035 - acc: 0.8517 - val_loss: 0.7322 - val_acc: 0.8456
Epoch 117/200
45000/45000 [==============================] - 18s 394us/step - loss: 0.6964 - acc: 0.8551 - val_loss: 0.7834 - val_acc: 0.8322
Epoch 118/200
45000/45000 [==============================] - 18s 396us/step - loss: 0.6893 - acc: 0.8582 - val_loss: 0.7476 - val_acc: 0.8372
Epoch 119/200
45000/45000 [==============================] - 18s 395us/step - loss: 0.7025 - acc: 0.8539 - val_loss: 0.8115 - val_acc: 0.8260
Epoch 120/200
45000/45000 [==============================] - 18s 398us/step - loss: 0.7009 - acc: 0.8542 - val_loss: 0.7297 - val_acc: 0.8512
Epoch 121/200
45000/45000 [==============================] - 18s 398us/step - loss: 0.6975 - acc: 0.8568 - val_loss: 0.7683 - val_acc: 0.8330
Epoch 122/200
45000/45000 [==============================] - 18s 396us/step - loss: 0.6956 - acc: 0.8561 - val_loss: 0.7538 - val_acc: 0.8376
Epoch 123/200
45000/

Epoch 174/200
45000/45000 [==============================] - 18s 399us/step - loss: 0.6968 - acc: 0.8552 - val_loss: 0.7722 - val_acc: 0.8318
Epoch 175/200
45000/45000 [==============================] - 18s 400us/step - loss: 0.6933 - acc: 0.8548 - val_loss: 0.7563 - val_acc: 0.8386
Epoch 176/200
45000/45000 [==============================] - 18s 395us/step - loss: 0.6965 - acc: 0.8542 - val_loss: 0.7529 - val_acc: 0.8358
Epoch 177/200
45000/45000 [==============================] - 18s 398us/step - loss: 0.6971 - acc: 0.8546 - val_loss: 0.7164 - val_acc: 0.8496
Epoch 178/200
45000/45000 [==============================] - 18s 397us/step - loss: 0.6962 - acc: 0.8540 - val_loss: 0.6946 - val_acc: 0.8584
Epoch 179/200
45000/45000 [==============================] - 18s 398us/step - loss: 0.7007 - acc: 0.8549 - val_loss: 0.7209 - val_acc: 0.8482
Epoch 180/200
45000/45000 [==============================] - 18s 395us/step - loss: 0.6925 - acc: 0.8569 - val_loss: 0.6994 - val_acc: 0.8528
Epoch 

In [30]:
import functools
import keras
top1_acc = functools.partial(keras.metrics.top_k_categorical_accuracy, k=1) #top-1精度

top1_acc.__name__ = 'top1_acc'

model_1.compile(loss='categorical_crossentropy',
             optimizer='adam',metrics=['accuracy','top_k_categorical_accuracy',top1_acc])

model_1.evaluate(x_test,y_test,verbose=0)

[0.767930428981781, 0.8359, 0.9914, 0.8359]

In [31]:
score=model_1.evaluate(x_test,y_test,verbose=0)
print("测试损失为：{:.4f}".format(score[0]))
print("测试准确率为：{:.4f}".format(score[1]))

测试损失为：0.7679
测试准确率为：0.8359
